Connect to SQLIte database

In [ ]:
# Import required packages
from langchain_classic.utilities import SQLDatabase

In [ ]:
# Create an SQLDatabase instance that connected to my SQLite db
db = SQLDatabase.from_uri(database_uri="sqlite:///../mySql.db") # type: ignore

In [ ]:
# a. Run a raw SQL command/query
query = "SELECT * FROM orders LIMIT 2;"
result = db.run(query)
print("Query Result:", result)


In [ ]:
# b. Get full database info, including tables, columns, datatypes, and example rows
db_info = db.get_table_info()
print("Database Info:")
print(db_info)

In [ ]:
# Load env variables
from langchain_openai import ChatOpenAI
import dotenv
import os

dotenv.load_dotenv()

In [ ]:
base_url = os.getenv("OPENAI_BASE_URL", default="")
api_key = os.getenv("OPENAI_API_KEY", default="")
model_name = os.getenv("OPENAI_MODEL_NAME", default="")

In [ ]:
# Create LLM
from pydantic import SecretStr

llm = ChatOpenAI(model=model_name, base_url=base_url, api_key=SecretStr(api_key))

In [ ]:
# System message
systemMessage = """
    # ROLE:
    - You are a highly intelligent SQL assistant.
    - Your task is to generate precise and efficient SQL queries based on the provided database schema and user requirements.

    # OUTPUT:
    - Return only the SQL query without any explanation, comments, or additional text.
    - Ensure the SQL query is optimized and adheres to best practices.

    # NOTE:
    - If the user request is unclear or ambiguous, request clarification.
"""
messages = [{"role": "system", "content": systemMessage}]

In [ ]:
query = "Give me all the products name and description and order ID from the highest amount order that placed on the month of october 2025"
schemas = db.get_table_info()

messages.append(
    {
        "role": "user",
        "content": f"""
            User Query: {query}
            Database Schema: {schemas}
        """,
    }
)

response = llm.invoke(messages)
print(response.content)

In [ ]:
db.run(response.content)

In [ ]:
query = "Give me all the products name and description and order ID, product count, customer name, and total amount placed on the month of october 2025"
schemas = db.get_table_info()

messages.append(
    {
        "role": "user",
        "content": f"""
            User Query: {query}
            Database Schema: {schemas}
        """,
    }
)

response = llm.invoke(messages)
print(response.content)

In [ ]:
result = db.run(response.content)
print(result)